In [1]:
# import libraries needed for the lesson
from pydantic import BaseModel, ValidationError, EmailStr
import json

In [2]:
# Create a Pydantic model for validating customer support input
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str
    

In [3]:
# Create a model instance
user_input = UserInput(
    name = "Joe User",
    email = "joe.user@example.com",
    query = "I forgot my password"
)
print(user_input)

name='Joe User' email='joe.user@example.com' query='I forgot my password'


In [7]:
# Create another model instance with an invalid email
user_input = UserInput(
    name = "Joe User",
    email = "not-an@email.c",# This format is what pydantic think a valid email address format
    query = "I forgot my password"
)
print(user_input)

name='Joe User' email='not-an@email.c' query='I forgot my password'


In [8]:
# Define a function to handle customer support input validation safely
def validate_user_input(input_data):
    try:
        # Attempt to create a UserInput model instance from input data
        user_input = UserInput(**input_data)
        print(f"✅ Valid user input created:")
        print(f"{user_input.model_dump_json(indent=2)}")
        return user_input
    except ValidationError as e:
        # Capture and display validation erros in a readable format
        print(f"❌ Validation error occurred:")
        for error in e.errors():
            print(f"    - {error['loc'][0]}: {error['msg']}")
            return None
    

In [10]:
# Create a dictionary with missing query
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I forgot my password"
}
# Validate the customer support data
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I forgot my password"
}


In [12]:
# Create a dictionary with missing query
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com"
}
# Validate the customer support data
user_input = validate_user_input(input_data)

❌ Validation error occurred:
    - query: Field required


Create optional features for the user input

In [19]:
# Import additional libraries for enhanced validation
from pydantic import Field
from typing import Optional
from datetime import date

# Enhance UserInput model with optional fields
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        None,
        description="5-digit order number (cannot start with 0)",
        ge=10000,
        le=99999
    )
    purchase_date: Optional[date] = None

In [20]:
# Create a dictionary with requried fields only
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I forgot my password"
}
# Validate the customer support data
user_input = validate_user_input(input_data)
# null is the json representation of None

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I forgot my password",
  "order_id": null,
  "purchase_date": null
}


In [21]:
user_input

UserInput(name='Joe User', email='joe.user@example.com', query='I forgot my password', order_id=None, purchase_date=None)

In [22]:
# Define a dictionalry with all fields including optional ones
input_data = {
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": f""" I bought a laptop carrying case and it turned out to be the wrong size. 
                    I need to return it. """,
    "order_id": 12345,
    "purchase_date": date(2025, 12, 31)
    
}
# Validate the customer support data
user_input = validate_user_input(input_data)


✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": " I bought a laptop carrying case and it turned out to be the wrong size. \n                    I need to return it. ",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [23]:
user_input

UserInput(name='Joe User', email='joe.user@example.com', query=' I bought a laptop carrying case and it turned out to be the wrong size. \n                    I need to return it. ', order_id=12345, purchase_date=datetime.date(2025, 12, 31))

In [24]:
# Define a dictionalry with all fields including optional ones
input_data = {
    "name": 121345,
    "email": "joe.user@example.com",
    "query": f""" I bought a laptop carrying case and it turned out to be the wrong size. 
                    I need to return it. """,
    "order_id": 12345,
    "purchase_date": date(2025, 12, 31)
    
}
# Validate the customer support data
user_input = validate_user_input(input_data)


❌ Validation error occurred:
    - name: Input should be a valid string


In [29]:
# This simulates Json data coming from a front-end application

json_data = """ 
{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I bought a keyboard and mouse and was overcharged",
    "order_id": 12345,
    "purchase_date":"2025-12-31"
}
"""

# Parse the Json string into a python dictionary
input_data = json.loads(json_data)
print("Parsed JSON", input_data)

Parsed JSON {'name': 'Joe User', 'email': 'joe.user@example.com', 'query': 'I bought a keyboard and mouse and was overcharged', 'order_id': 12345, 'purchase_date': '2025-12-31'}


In [30]:
# validate the customer support data from json
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a keyboard and mouse and was overcharged",
  "order_id": 12345,
  "purchase_date": "2025-12-31"
}


In [37]:
# This simulates Json data coming from a front-end application

json_data = """ 
{
    "name": "Joe User",
    "email": "joe.user@example.com",
    "query": "I bought a keyboard and mouse and was overcharged",
    "order_id": "11234",
    "purchase_date":"2025-12-31"
}
"""

# Parse the Json string into a python dictionary
input_data = json.loads(json_data)
print("Parsed JSON", input_data)

Parsed JSON {'name': 'Joe User', 'email': 'joe.user@example.com', 'query': 'I bought a keyboard and mouse and was overcharged', 'order_id': '11234', 'purchase_date': '2025-12-31'}


In [38]:
# validate the customer support data from json
user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a keyboard and mouse and was overcharged",
  "order_id": 11234,
  "purchase_date": "2025-12-31"
}


In [39]:
user_input = UserInput.model_validate_json(json_data)
print(user_input.model_dump_json(indent=2))

{
  "name": "Joe User",
  "email": "joe.user@example.com",
  "query": "I bought a keyboard and mouse and was overcharged",
  "order_id": 11234,
  "purchase_date": "2025-12-31"
}
